# **Part One**
evaluating cosin similarity without indexing:

In [1]:
! pip install sentence-transformers scikit-learn

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
  

In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import math

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy import spatial

from sentence_transformers import SentenceTransformer
from sentence_transformers import  util

import scipy.spatial

import pandas as pd
import numpy as np


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [44]:
from datasets import load_dataset
sentences = load_dataset("Omartificial-Intelligence-Space/Arabic-NLi-Triplet")

In [45]:
from collections import OrderedDict
sentences=sentences['train']['positive']
sentences = OrderedDict.fromkeys(sentences)
sentences= list(sentences.keys())

In [7]:
model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [8]:

def preprocess_query(query):
    # Tokenization, stop word removal, and stemming
    stop_words = set(stopwords.words('arabic'))
    stemmer = PorterStemmer()
    tokens = query.lower().split()
    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)


In [39]:

def hybrid_search(query, top_k=10, weight_st=0.5, weight_tfidf=0.5):
    # Encode the query using SentenceTransformers
    query_preprocessed = preprocess_query(query)
    query_embedding = model.encode([query_preprocessed], convert_to_tensor=True)

    # Compute cosine similarities between the query and the documents using SentenceTransformers
    cosine_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

    # Transform the query using the TF-IDF vectorizer
    query_tfidf = tfidf_vectorizer.transform([query_preprocessed])

    # Calculate cosine similarity between the query and the documents using TF-IDF
    tfidf_similarities = cosine_similarity(query_tfidf, tfidf_matrix)[0]
    #print(tfidf_similarities)

    # Combine the scores with weights
    combined_scores = (weight_st * cosine_scores.cpu().numpy() + weight_tfidf * tfidf_similarities[:1000]) / (weight_st + weight_tfidf)
    #print(combined_scores)

    # Normalize scores (optional)
    combined_scores = (combined_scores - np.min(combined_scores)) / (np.max(combined_scores) - np.min(combined_scores))

    # Find the top_k highest scores
    top_results = np.argpartition(-combined_scores, range(top_k))[:top_k]
    #print(top_results)

    print(f"Query: {query}\nTop {top_k} most similar arabic_documents:")
    for idx in top_results:
        print(f"Document: {sentences[idx]}, Combined Score: {combined_scores[idx]:.4f}")

    return top_results, combined_scores



In [46]:

# Encode the documents using SentenceTransformers
document_embeddings = model.encode(sentences[:1000], convert_to_tensor=True)

# Create a TF-IDF Vectorizer and fit on the documents
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)


In [47]:
user_query = input("Enter your query :")

Enter your query :اللعب مع الاصدقاء


In [37]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [48]:
%%time
hybrid_search(user_query)
predicted_ranks, _ = hybrid_search(user_query)

Query: اللعب مع الاصدقاء
Top 10 most similar arabic_documents:
Document: أطفال يلعبون في ملعب., Combined Score: 1.0000
Document: الأطفال يلعبون في الخارج., Combined Score: 0.9276
Document: أربعة أصدقاء يستمتعون في الشاطئ في يوم حار., Combined Score: 0.9246
Document: ثلاثة جراء يلعبون في الخارج., Combined Score: 0.8622
Document: زوجان يلعبان مع طفل صغير في الخارج, Combined Score: 0.8570
Document: إمرأتان تعانقان بعضهما البعض, Combined Score: 0.8496
Document: مجموعة من الأصدقاء يرتاحون ويستمتعون بيوم مشمس جميل في الحديقة, Combined Score: 0.8417
Document: زوجين يمشون, Combined Score: 0.8370
Document: أرض جميلة والناس يلعبون, Combined Score: 0.7930
Document: هناك فتاتين تلعبان بالخارج, Combined Score: 0.7702
Query: اللعب مع الاصدقاء
Top 10 most similar arabic_documents:
Document: أطفال يلعبون في ملعب., Combined Score: 1.0000
Document: الأطفال يلعبون في الخارج., Combined Score: 0.9276
Document: أربعة أصدقاء يستمتعون في الشاطئ في يوم حار., Combined Score: 0.9246
Document: ثلاثة جراء يلعبون ف

#**Part Two**
searching with index

In [50]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.1 MB/s eta 0:00:00


In [51]:
import faiss

In [52]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
#model = SentenceTransformer('bert-base-nli-mean-tokens') #encoding model
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [53]:
sentence_embeddings = model.encode(sentences[:1000])
sentence_embeddings.shape

(1000, 384)

In [58]:
k = 10
xq = model.encode(["اللعب مع الأصدقاء"])

In [54]:
#IndexFlatL2
dimension = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

In [55]:
index.is_trained

True

In [56]:
index.add(sentence_embeddings)

In [57]:
index.ntotal

1000

In [59]:
%%time
D, I = index.search(xq, k)  # search xq is query k n.output
print(I)

[[730 135 759   8 123 980 126  42 961 562]]
CPU times: user 1.27 ms, sys: 1.02 ms, total: 2.29 ms
Wall time: 1.54 ms


In [60]:
[f'{i}: {sentences[i]}' for i in I[0]]

['730: ثلاثة جراء يلعبون في الخارج.',
 '135: هناك بعض الناس بالخارج',
 '759: الناس بالخارج',
 '8: إمرأتان تعانقان بعضهما البعض',
 '123: الناس يمسكون أيدي بعضهم ويمشون',
 '980: الناس في الخارج',
 '126: الجوقة تغني في القداس',
 '42: أنثى بجانب رجل',
 '961: الناس يقومون باليوغا',
 '562: مجموعة من الناس بالخارج']

In [61]:
#Hierarchical Navigable Small World (HNSW) Index
dimension = sentence_embeddings.shape[1]
n_neighbors = 10
index = faiss.IndexHNSWFlat(dimension,n_neighbors)

In [62]:
index.is_trained

True

In [63]:
index.add(sentence_embeddings)

In [64]:
index.ntotal

1000

In [65]:
%%time
D, I = index.search(xq, k)  # search xq is query k n.output
print(I)

[[730 135 759   8 123 980 126  42 961 562]]
CPU times: user 2.1 ms, sys: 5 µs, total: 2.1 ms
Wall time: 1.78 ms


In [66]:
[f'{i}: {sentences[i]}' for i in I[0]]

['730: ثلاثة جراء يلعبون في الخارج.',
 '135: هناك بعض الناس بالخارج',
 '759: الناس بالخارج',
 '8: إمرأتان تعانقان بعضهما البعض',
 '123: الناس يمسكون أيدي بعضهم ويمشون',
 '980: الناس في الخارج',
 '126: الجوقة تغني في القداس',
 '42: أنثى بجانب رجل',
 '961: الناس يقومون باليوغا',
 '562: مجموعة من الناس بالخارج']

In [67]:
#PQ
dimension = sentence_embeddings.shape[1]
m = 8
nbits = 8
index = faiss.IndexPQ(dimension, m, nbits)

In [68]:
index.is_trained

False

In [69]:
index.train(sentence_embeddings)
index.is_trained

True

In [70]:
index.add(sentence_embeddings)

In [71]:
index.ntotal

1000

In [72]:
%%time
D, I = index.search(xq, k)  # search xq is query k n.output
print(I)

[[759 135 980 562 979 724 269 961 123 856]]
CPU times: user 2.64 ms, sys: 0 ns, total: 2.64 ms
Wall time: 1.94 ms


In [73]:
[f'{i}: {sentences[i]}' for i in I[0]]

['759: الناس بالخارج',
 '135: هناك بعض الناس بالخارج',
 '980: الناس في الخارج',
 '562: مجموعة من الناس بالخارج',
 '979: هناك مجموعة من الناس يقفون بالخارج',
 '724: الناس يلعبون رياضة تكريما للأشخاص المعاقين.',
 '269: هناك منحوتات قريبة.',
 '961: الناس يقومون باليوغا',
 '123: الناس يمسكون أيدي بعضهم ويمشون',
 '856: يبدو أن هؤلاء الشباب يغنون ويرقصون في الخارج']

In [75]:
#IVF
nlist = 100
quantizer = faiss.IndexFlatL2(dimension)
index = faiss.IndexIVFFlat(quantizer, dimension, nlist)

In [76]:
index.is_trained

False

In [77]:
index.train(sentence_embeddings)
index.is_trained

True

In [78]:
index.add(sentence_embeddings)
index.ntotal

1000

In [79]:
%%time
D, I = index.search(xq, k)  # search xq is query k n.output
print(I)

[[ 42 961 270 814 959 921 751 269 960   9]]
CPU times: user 2.5 ms, sys: 0 ns, total: 2.5 ms
Wall time: 2.45 ms


In [80]:
[f'{i}: {sentences[i]}' for i in I[0]]

['42: أنثى بجانب رجل',
 '961: الناس يقومون باليوغا',
 '270: رجل وامرأة يقفان',
 '814: فتيان وامرأة يقفان أمام كشك "برونتو كوبز هامبورجر".',
 '959: دروس اليوغا جارية',
 '921: الحلاق في العمل.',
 '751: إنهم بالخارج يرتدون معاطف',
 '269: هناك منحوتات قريبة.',
 '960: الناس امتدوا على سجادة اليوغا',
 '9: فريق يحاول تعقب هارب']